In [2]:
import numpy as np

In [44]:
n = 30
res = ((2 ** n) * 32)
res / (1000**3)

34.359738368

274.877906944

In [48]:
def zero_out_array(arr):
    arr.fill(0)

def contract_tensor(state, gate, qubit, shape):
    n = len(shape)
    total_elements = 2 ** n
    new_state = np.zeros(total_elements, dtype=complex)

    # Iterate over all possible indices of the state tensor
    for idx in range(total_elements):
        # print(f'idx {idx} #########################')
        new_idx = np.zeros(n, dtype=int)
        old_idx = np.zeros(n, dtype=int)
        temp = idx

        # Compute the multi-dimensional index
        for i in range(n-1, -1, -1):
            # print(f'i {i}')
            new_idx[i] = temp % shape[i]
            temp //= shape[i]
            # print(f'temp {temp}')
        # print(f'new_idx {new_idx} +++++++++++')

        # Perform the tensor contraction for the specified qubit
        # TODO: make the algorithm more generic to work with all dimensions (currently it is 2)
        for j in range(2):
            # print(f'j {j}')
            # Copy new_idx to old_idx
            old_idx[:] = new_idx[:]
            old_idx[qubit] = j

            # Compute the linear index for old_idx
            old_linear_idx = 0
            factor = 1
            for i in range(n-1, -1, -1):
                # print(f'i2 {i}')
                old_linear_idx += old_idx[i] * factor
                factor *= shape[i]
            # print(f'old_linear_idx {old_linear_idx} +++++++++++')

            new_state[idx] += gate[new_idx[qubit], j] * state[old_linear_idx]
            # print(f'new_state {new_state}')

    return new_state


In [ ]:
q0 = np.array([1, 0])
q1 = np.array([1, 0])

n = 30
N = 2**n
shape = [2] * n

# Define the Hadamard gate
H = np.array([[1, 1], [1, -1]], dtype=complex) / np.sqrt(2)

# Define the state vector
state = np.array([1] + [0]*(N-1))

print(state)
new_state = contract_tensor(state, H, 0, shape)
print(new_state)

# new_state = contract_tensor(new_state, H, 1, shape)
# print(new_state)

# new_state = contract_tensor(new_state, H, 2, shape)
# print(new_state)

[1 0 0 0 0 0 0 0]
[0.70710678+0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.70710678+0.j 0.        +0.j 0.        +0.j 0.        +0.j]


KeyboardInterrupt: 

In [4]:
n = 10
N = 2**n
# state = np.array([1] + [0]*(N-1))

1 / np.sqrt(N)


0.03125

In [15]:

def tensor_contract(tensor1, tensor2, axes):
    """
    Perform tensor contraction between two tensors along specified axes.

    Args:
        tensor1: The first tensor.
        tensor2: The second tensor.
        axes: A tuple of two lists, specifying the axes to contract over.
              axes[0] are the axes of tensor1, and axes[1] are the axes of tensor2.

    Returns:
        A new tensor resulting from the contraction.
    """
    # Validate input
    if len(axes[0]) != len(axes[1]):
        raise ValueError("Axes lengths must be the same")

    # Move the specified axes to the end for tensor1 and the start for tensor2
    tensor1 = np.moveaxis(tensor1, axes[0], range(-len(axes[0]), 0))
    tensor2 = np.moveaxis(tensor2, axes[1], range(len(axes[1])))

    # Reshape tensors for matrix multiplication
    tensor1_shape = tensor1.shape
    tensor2_shape = tensor2.shape
    tensor1_reshaped = tensor1.reshape(-1, np.prod(tensor1_shape[-len(axes[0]):]))
    tensor2_reshaped = tensor2.reshape(np.prod(tensor2_shape[:len(axes[1])]), -1)

    # Perform matrix multiplication
    result = np.dot(tensor1_reshaped, tensor2_reshaped)

    # Reshape the result to the proper shape
    result_shape = tensor1_shape[:-len(axes[0])] + tensor2_shape[len(axes[1]):]
    result = result.reshape(result_shape)

    return result

# Example usage
tensor1 = np.random.rand(3, 4, 5)
tensor2 = np.random.rand(5, 2, 6)
axes = ([2], [0])  # Contract over the third axis of tensor1 and the first axis of tensor2

result = tensor_contract(tensor1, tensor2, axes)
print("Resulting tensor shape:", result.shape)


Resulting tensor shape: (3, 4, 2, 6)


In [68]:
def apply_hadamard_to_single_qubit(state, H, qubit):
    state_shape = state.shape

    # Initialize a new state tensor to hold the result
    new_shape = list(state_shape)
    new_state = np.zeros(new_shape, dtype=complex)

    # Iterate over all possible indices of the state tensor
    for idx in np.ndindex(state_shape):
        print('idx: ', idx)
        new_idx = list(idx)
        print('new_idx ', new_idx)
        # Perform the tensor contraction manually for the specified qubit
        new_value = 0
        for j in range(2):
            old_idx = list(idx)
            old_idx[qubit] = j
            new_value += H[new_idx[qubit], j] * state[tuple(old_idx)]
        new_state[tuple(new_idx)] = new_value

    return new_state



def contract_tensor(state, gate, qubit):
    state_shape = state.shape

    # Initialize a new state tensor to hold the result
    new_shape = list(state_shape)
    new_state = np.zeros(new_shape, dtype=complex)

    # Iterate over all possible indices of the state tensor
    for idx in np.ndindex(state_shape):
        # print('idx: ', idx)
        new_idx = list(idx)
        # print('new_idx ', new_idx)
        # Perform the tensor contraction manually for the specified qubit
        for j in range(2):
            old_idx = list(idx)
            old_idx[qubit] = j
            new_state[tuple(new_idx)] += gate[new_idx[qubit], j] * state[tuple(old_idx)]
    return new_state

In [69]:
q0 = np.array([1, 0])
q1 = np.array([1, 0])

# Define the Hadamard gate
H = np.array([[1, 1], [1, -1]], dtype=complex) / np.sqrt(2)

# Define the state vector
state = np.outer(q0, q1).reshape([2]* 2)

s = contract_tensor(state, H, 0)
print(s)
s = contract_tensor(s, H, 1)
s

[[0.70710678+0.j 0.        +0.j]
 [0.70710678+0.j 0.        +0.j]]


array([[0.5+0.j, 0.5+0.j],
       [0.5+0.j, 0.5+0.j]])

[[0.70710678+0.j 0.        +0.j]
 [0.70710678+0.j 0.        +0.j]]


array([[0.5+0.j, 0.5+0.j],
       [0.5+0.j, 0.5+0.j]])

In [54]:
s = tensor_contract(state, H, ([0], [0]))
print(s)
s = tensor_contract(s, H, ([0], [1]))
s

[[0.70710678+0.j 0.70710678+0.j]
 [0.        +0.j 0.        +0.j]]


array([[ 1.00000000e+00+0.j, -2.23711432e-17+0.j],
       [ 0.00000000e+00+0.j,  0.00000000e+00+0.j]])

In [59]:
import numpy as np

def tensor_contract(tensor1, tensor2, axes):
    """
    Perform tensor contraction between two tensors along specified axes.

    Args:
        tensor1: The first tensor.
        tensor2: The second tensor.
        axes: A tuple of two lists, specifying the axes to contract over.
              axes[0] are the axes of tensor1, and axes[1] are the axes of tensor2.

    Returns:
        A new tensor resulting from the contraction.
    """
    # Ensure axes lists have the same length
    if len(axes[0]) != len(axes[1]):
        raise ValueError("Axes lengths must be the same")

    # Calculate the shape of the resulting tensor
    new_shape = list(tensor1.shape[:axes[0][0]]) + list(tensor1.shape[axes[0][0]+1:]) + \
                list(tensor2.shape[:axes[1][0]]) + list(tensor2.shape[axes[1][0]+1:])

    # Initialize the resulting tensor
    result = np.zeros(new_shape, dtype=complex)

    # Iterate over all indices of the non-contracted dimensions
    for idx1 in np.ndindex(*tensor1.shape[:axes[0][0]], *tensor1.shape[axes[0][0]+1:]):
        print(idx1)
        for idx2 in np.ndindex(*tensor2.shape[:axes[1][0]], *tensor2.shape[axes[1][0]+1:]):
            print(idx2)
            contracted_sum = 0
            for k in range(tensor1.shape[axes[0][0]]):
                index1 = idx1[:axes[0][0]] + (k,) + idx1[axes[0][0]:]
                index2 = idx2[:axes[1][0]] + (k,) + idx2[axes[1][0]:]
                contracted_sum += tensor1[index1] * tensor2[index2]
            result[idx1 + idx2] = contracted_sum

    return result

# Define the initial qubit states
q0 = np.array([1, 0])
q1 = np.array([1, 0])

# Define the Hadamard gate
H = np.array([[1, 1], [1, -1]], dtype=complex) / np.sqrt(2)

# Define the state vector for 2 qubits (|00>)
initial_state = np.kron(q0, q1).reshape(2, 2)
initial_state = np.kron(np.kron(q0, q1), q0).reshape(2, 2, 2)


# Apply the Hadamard gate to the first qubit
state_after_first_H = tensor_contract(H, initial_state, ([1], [0]))

# Apply the Hadamard gate to the second qubit
state_after_second_H = tensor_contract(H, state_after_first_H, ([1], [1]))

# Reshape the result to a vector
final_state = state_after_second_H.reshape(8)

print("Resulting state vector after applying H to both qubits:")
print(final_state)


(0,)
(0, 0)
(0, 1)
(1, 0)
(1, 1)
(1,)
(0, 0)
(0, 1)
(1, 0)
(1, 1)
(0,)
(0, 0)
(0, 1)
(1, 0)
(1, 1)
(1,)
(0, 0)
(0, 1)
(1, 0)
(1, 1)
Resulting state vector after applying H to both qubits:
[0.5+0.j 0. +0.j 0.5+0.j 0. +0.j 0.5+0.j 0. +0.j 0.5+0.j 0. +0.j]
